In [4]:
# Campus Energy-Use Dashboard
# Capstone Project - Khushi
# B.Tech CSE (AI & ML)

import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime

# Create folders
os.makedirs("data", exist_ok=True)
os.makedirs("output", exist_ok=True)

print("Generating sample data for 4 buildings...")

n = 60

# 1. Admin Building
pd.DataFrame({
    "timestamp": pd.date_range("2024-01-01", periods=n, freq="4h"),
    "kwh": [35, 75, 98, 28] * (n//4)
}).to_csv("data/Admin_Building.csv", index=False)

# 2. Library
pd.DataFrame({
    "timestamp": pd.date_range("2024-01-01 08:00", periods=n, freq="5h"),
    "kwh": [48, 95, 118, 72] * (n//4)
}).to_csv("data/Library.csv", index=False)

# 3. Engineering Block
pd.DataFrame({
    "timestamp": pd.date_range("2024-01-01 07:00", periods=n, freq="3h"),
    "kwh": [85, 155, 178, 135] * (n//4)
}).to_csv("data/Engineering_Block.csv", index=False)

# 4. Hostel A
pd.DataFrame({
    "timestamp": pd.date_range("2024-01-01", periods=n, freq="6h"),
    "kwh": [140, 98, 115, 175, 205, 135] * (n//6)
}).to_csv("data/Hostel_A.csv", index=False)

print("All 4 files created!")

# Load all data
all_data = []
for file in os.listdir("data"):
    if file.endswith(".csv"):
        name = file.replace(".csv", "").replace("_", " ").title()
        df = pd.read_csv(f"data/{file}", parse_dates=["timestamp"])
        df["building"] = name
        all_data.append(df)
        print(f"Loaded {len(df)} records from {name}")

df = pd.concat(all_data, ignore_index=True)
df = df.sort_values("timestamp").reset_index(drop=True)

# Simple OOP
class Building:
    def __init__(self, name):
        self.name = name
        self.readings = []
    def add(self, kwh):
        self.readings.append(kwh)
    def total(self):
        return sum(self.readings)

buildings = {}
for _, row in df.iterrows():
    b = row["building"]
    if b not in buildings:
        buildings[b] = Building(b)
    buildings[b].add(row["kwh"])

total_campus = sum(b.total() for b in buildings.values())
highest = max(buildings.values(), key=lambda b: b.total())

# Dashboard
plt.figure(figsize=(16,10))
plt.suptitle("CAMPUS ENERGY DASHBOARD - KHUSHI", fontsize=20, fontweight='bold')

plt.subplot(2,2,1)
for b in df["building"].unique():
    temp = df[df["building"]==b].set_index("timestamp").resample("D")["kwh"].sum()
    plt.plot(temp.index, temp.values, marker="o", label=b)
plt.title("Daily Consumption Trend")
plt.ylabel("kWh")
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(2,2,2)
df.groupby("building")["kwh"].mean().sort_values().plot(kind="barh", color="skyblue", ax=plt.gca())
plt.title("Average Usage per Building")
plt.xlabel("kWh")

plt.subplot(2,2,3)
peaks = df.loc[df.groupby("building")["kwh"].idxmax()]
plt.scatter(peaks["timestamp"], peaks["kwh"], c="red", s=100)
for i in peaks.index:
    plt.annotate(peaks.loc[i,"building"], (peaks.loc[i,"timestamp"], peaks.loc[i,"kwh"]), xytext=(5,5), textcoords="offset points")
plt.title("Peak Load Events")

plt.subplot(2,2,4)
df.set_index("timestamp").resample("W")["kwh"].sum().plot(kind="bar", color="lightgreen", ax=plt.gca())
plt.title("Weekly Campus Total")
plt.ylabel("kWh")
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig("output/dashboard.png", dpi=300, bbox_inches="tight")
plt.close()

# Export files
df.to_csv("output/cleaned_energy_data.csv", index=False)
df.groupby("building")["kwh"].agg(["sum","mean","max","min"]).round(2).to_csv("output/building_summary.csv")

# Summary report - NO SPECIAL CHARACTERS
report = """
CAMPUS ENERGY CONSUMPTION REPORT
---------------------------------

Generated on           : {date}
Total Campus Consumption : {total:,.2f} kWh
Highest Consuming Building : {highest}
Peak Hours             : 6 PM - 10 PM
Recommendation         : Install motion sensors in Hostel A

Prepared by:
Khushi
B.Tech CSE (AI & ML)
""".format(
    date=datetime.now().strftime("%d %B %Y, %I:%M %p"),
    total=total_campus,
    highest=highest.name
)

with open("output/summary.txt", "w", encoding="utf-8") as f:
    f.write(report.strip())

print("\n" + report)

Generating sample data for 4 buildings...
All 4 files created!
Loaded 60 records from Admin Building
Loaded 35064 records from Building A
Loaded 35064 records from Building B
Loaded 35064 records from Building C
Loaded 60 records from Engineering Block
Loaded 60 records from Hostel A
Loaded 60 records from Library


CAMPUS ENERGY CONSUMPTION REPORT
---------------------------------

Generated on           : 02 December 2025, 12:33 AM
Total Campus Consumption : 1,297,001,721.98 kWh
Highest Consuming Building : Building B
Peak Hours             : 6 PM - 10 PM
Recommendation         : Install motion sensors in Hostel A

Prepared by:
Khushi
B.Tech CSE (AI & ML)

